<h3>Vytvoření y</h3>

y = co má neuronka předpovídat. V našem případě začneme tím, zda byl v daném čase L-mod nebo H-mod.

Je třeba:

- načíst signály popisující začátek a konec H-módu v daném výboji: `t_H_mode_start` a `t_H_mode_end`.

- z adresáře s obrazovými daty zjistit časy snímků kamery v daném výboji

- pro každý čas zjistit, zda jde o L-mód nebo H-mód

- uložit do csv (např ve formátu slupců [název souboru se snímkem z kamery, čas ve výboji, label zda L-mode/H-mode]) - použijte pandas

Importing libraries

In [9]:
import os
os.chdir("/compass/Shared/Users/bogdanov/vyzkumny_ukol")

In [10]:
from cdb_extras import xarray_support as cdbxr   # načítání dat z databáze COMPASSu
from pyCDB import client
import numpy as np         # práce s numerickými poli
import pandas as pd
from pathlib import Path
%matplotlib inline
from tqdm.auto import tqdm


#import module with functions from the tutorial
import cdb_img_processing as img
cdb = client.CDBClient()

Importing data

In [11]:
shot = 16773 # 18057]
t_ELM_start = cdb.get_signal(f"t_ELM_start/SYNTHETIC_DIAGNOSTICS:{shot}:bogdanov_2023")
t_ELM_end = cdb.get_signal(f"t_ELM_end/SYNTHETIC_DIAGNOSTICS:{shot}:bogdanov_2023")
t_H_mode_start = cdb.get_signal(f"t_H_mode_start/SYNTHETIC_DIAGNOSTICS:{shot}:bogdanov_2023")
t_H_mode_end = cdb.get_signal(f"t_H_mode_end/SYNTHETIC_DIAGNOSTICS:{shot}:bogdanov_2023")

In [12]:
ris1_data = img.load_RIS_data(shot, 1)
ris2_data = img.load_RIS_data(shot, 2)

Saving the images with plasma on it ()

In [13]:
out_path = Path('./imgs')
ris1_names = img.save_ris_images_to_folder(ris1_data, path=out_path, ris=1, shot=shot, use_discharge_duration=True, just_names=False)
ris2_names = img.save_ris_images_to_folder(ris1_data, path=out_path, ris=2, shot=shot, use_discharge_duration=True, just_names=False)

Demosaicing images


Saving images:   0%|          | 0/2180 [00:00<?, ?it/s]

Create a `pd.DataFrame` containing time, mode, and image path

In [14]:
LorH = pd.DataFrame(data={'mode':np.full(len(ris1_data), 'L-mode')}, index=pd.Index(ris1_data.time, name='time'))#contains time and the state of the plasma (L-mode, H-mode, ELM)

In [15]:
#TODO:  To create a DataFrame with only one row, you need to specify an index, so if during the shot plasma was in H-mode more than once then next row creates different shape
t_H_mode = pd.DataFrame({'start':t_H_mode_start.data, 'end':t_H_mode_end.data}, index=[0])
t_ELM = pd.DataFrame({'start':t_ELM_start.data, 'end':t_ELM_end.data})


for H_mode in t_H_mode.values:
    LorH.loc[H_mode[0]:H_mode[1]] = 'H-mode'

for elm in t_ELM.values:
    LorH.loc[elm[0]:elm[1]] = 'ELM'

#Discarding pictures without plasma
discharge_start, discharge_end = img.discharge_duration(shot)
LorH = LorH[discharge_start : discharge_end]

#Appending columns with paths of the RIS imgs
LorH['filename'] = np.array(ris1_names)



LorH.to_csv(f'/compass/Shared/Users/bogdanov/vyzkumny_ukol/LHmode-detection-shot{shot}.csv')

In [16]:
LorH.head()

,mode,filename
time,,
960.4,L-mode,imgs/RIS1_16773_t=960.4000000000001.png
960.6,L-mode,imgs/RIS1_16773_t=960.6000000000001.png
960.8,L-mode,imgs/RIS1_16773_t=960.8000000000001.png
961.0,L-mode,imgs/RIS1_16773_t=961.0000000000001.png
961.2,L-mode,imgs/RIS1_16773_t=961.2000000000002.png
